In [2]:
#!pip install catboost
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# CSV 파일 읽기
train = pd.read_csv('train_for_eda.csv')
test = pd.read_csv('test_for_eda(notnull).csv')

# 예를 들어 target 컬럼이 'target'이라고 가정
X_train = train.drop(columns=['target'])
y_train = train['target']
X_test = test  # 테스트 데이터에는 타겟 값이 없다고 가정

# 검증용 데이터 분리 (optional)
from sklearn.model_selection import train_test_split
X_train_sub, X_valid, y_train_sub, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


/usr/local/lib/python3.8/dist-packages/xgboost/core.py:265: FutureWarning: Your system has an old version of glibc (< 2.28). We will stop supporting Linux distros with glibc older than 2.28 after **May 31, 2025**. Please upgrade to a recent Linux distro (with glibc 2.28+) to use future versions of XGBoost.
Note: You have installed the 'manylinux2014' variant of XGBoost. Certain features such as GPU algorithms or federated learning are not available. To use these features, please upgrade to a recent Linux distro with glibc 2.28+, and install the 'manylinux_2_28' variant.
  warnings.warn(


In [4]:
train['계약년월']

0         202306
1         202301
2         202302
3         202304
4         202304
           ...  
155918    202008
155919    202001
155920    202001
155921    202003
155922    202007
Name: 계약년월, Length: 155923, dtype: int64

In [34]:
print(X_train.columns)
print(X_valid.columns)


Index(['아파트명', '전용면적(㎡)', '계약년월', '계약일', '층', '건축년도', '구', '회사채금리', '매매가격지수',
       '건설공사비지수', '버스정류장수', '지하철수', '강남권여부', '계약년', '계약월', '연식'],
      dtype='object')
Index(['아파트명', '전용면적(㎡)', '계약년월', '계약일', '층', '건축년도', '구', '회사채금리', '매매가격지수',
       '건설공사비지수', '버스정류장수', '지하철수', '강남권여부', '계약년', '계약월', '연식'],
      dtype='object')


In [35]:
print(X_train['계약일'].isnull().sum())
print(X_valid['계약일'].isnull().sum())

0
0


In [36]:
print(X_train['계약일'].dtype)
print(X_valid['계약일'].dtype)

int64
int64


In [37]:
from catboost import Pool, CatBoostRegressor

cat_features = ['아파트명', '구', '층']  # 범주형 지정

print("Train data shape:", X_train.shape, y_train.shape)
print("Valid data shape:", X_valid.shape, y_valid.shape)

print("Train columns:", list(X_train.columns))
print("Valid columns:", list(X_valid.columns))

train_pool = Pool(X_train, y_train, cat_features=cat_features)
valid_pool = Pool(X_valid, y_valid, cat_features=cat_features)

cat_model = CatBoostRegressor(iterations=1000, learning_rate=0.1, depth=6, loss_function='RMSE', verbose=100)

cat_model.fit(train_pool, eval_set=valid_pool, early_stopping_rounds=50)

Train data shape: (155923, 16) (155923,)
Valid data shape: (31185, 16) (31185,)
Train columns: ['아파트명', '전용면적(㎡)', '계약년월', '계약일', '층', '건축년도', '구', '회사채금리', '매매가격지수', '건설공사비지수', '버스정류장수', '지하철수', '강남권여부', '계약년', '계약월', '연식']
Valid columns: ['아파트명', '전용면적(㎡)', '계약년월', '계약일', '층', '건축년도', '구', '회사채금리', '매매가격지수', '건설공사비지수', '버스정류장수', '지하철수', '강남권여부', '계약년', '계약월', '연식']
0:	learn: 64686.8400991	test: 64122.9678313	best: 64122.9678313 (0)	total: 23ms	remaining: 23s
100:	learn: 23760.9438226	test: 22268.5805783	best: 22268.5805783 (100)	total: 1.72s	remaining: 15.3s
200:	learn: 20158.9577821	test: 18804.8392046	best: 18804.8392046 (200)	total: 3.41s	remaining: 13.6s
300:	learn: 18288.7264748	test: 17010.3726117	best: 17010.3726117 (300)	total: 5.16s	remaining: 12s
400:	learn: 17082.5888240	test: 15906.5676174	best: 15906.5676174 (400)	total: 6.93s	remaining: 10.4s
500:	learn: 16184.0206308	test: 15063.3126220	best: 15063.3126220 (500)	total: 8.61s	remaining: 8.58s
600:	learn: 15547.4315248	t

In [51]:
X_train_sub, X_valid, y_train_sub, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

cat_features = ['아파트명', '구', '층']

train_pool = Pool(X_train, y_train, cat_features=cat_features)
valid_pool = Pool(X_valid, y_valid, cat_features=cat_features)

model = CatBoostRegressor(iterations=1000, learning_rate=0.1, loss_function='RMSE', verbose=100)

model.fit(train_pool, eval_set=valid_pool, early_stopping_rounds=50)

preds = model.predict(valid_pool)
rmse = np.sqrt(mean_squared_error(y_valid, preds))
print("Validation RMSE:", rmse)

# 반올림하여 정수로 변환
preds_int = np.round(preds).astype(int)

# submission에 정수로 저장
submission = pd.DataFrame({
    'target': preds_int
})

submission.to_csv('submission_add_yongsan.csv', index=False)


0:	learn: 64686.8400991	test: 64122.9678313	best: 64122.9678313 (0)	total: 21.5ms	remaining: 21.5s
100:	learn: 23760.9438226	test: 22268.5805783	best: 22268.5805783 (100)	total: 1.72s	remaining: 15.3s
200:	learn: 20158.9577821	test: 18804.8392046	best: 18804.8392046 (200)	total: 3.43s	remaining: 13.7s
300:	learn: 18288.7264748	test: 17010.3726117	best: 17010.3726117 (300)	total: 5.23s	remaining: 12.1s
400:	learn: 17082.5888240	test: 15906.5676174	best: 15906.5676174 (400)	total: 7.01s	remaining: 10.5s
500:	learn: 16184.0206308	test: 15063.3126220	best: 15063.3126220 (500)	total: 8.73s	remaining: 8.7s
600:	learn: 15547.4315248	test: 14494.5596061	best: 14494.5596061 (600)	total: 10.4s	remaining: 6.93s
700:	learn: 14996.1726612	test: 13997.8494042	best: 13997.8494042 (700)	total: 12.2s	remaining: 5.21s
800:	learn: 14553.6994481	test: 13599.9676731	best: 13599.9676731 (800)	total: 14s	remaining: 3.48s
900:	learn: 14129.2990987	test: 13197.7766784	best: 13197.7766784 (900)	total: 15.8s	rem